In [9]:
dark_current = 0.005    #  [e-/px/s]  Dark current of the camera
exposure_time =  1      #  [s] Exposure time in seconds
max_photons = 1     # Maximum number of photons
rep_count = 100     # number of repetitions for generating em
alpha = 0.01    # probability of electron multiplying
N = 2   #number of gain stages
sigma_read = 0.005     # S.D of read noise
g_EM = 32   # EM gain

In [ ]:
import torch
import numpy as np
cam_dYdt_dark = .005
cam_t_exp = 1

def noise_model(det_Y):

    # Dark Noise Model
    Ydark   = cam_dYdt_dark * cam_t_exp
    Yhat        = torch.poisson(det_Y + Ydark)

    

In [11]:
def f_genEmhist(max_photons, rep_count,N,alpha):
    in_photons = np.arange(1, max_photons + 1)

    emhist_100 = np.tile(in_photons, (100, 1)).T
    emhist = [emhist_100.copy() for _ in range(rep_count // 100)]

    def simulate_gain_stage(emhist_slice, alpha):
        for _ in range(N):
            emhist_slice += np.random.binomial(emhist_slice, alpha)

    def process_slice(args):
        emhist_slice, alpha = args
        simulate_gain_stage(emhist_slice, alpha)
        return emhist_slice

    for i in tqdm(range(len(emhist))):
        emhist_slice = emhist[i]
        args = (emhist_slice, alpha)

    emhist = np.concatenate(emhist, axis=1)

    return emhist
